## 대가리 수정 부분입니다

기존의 코드를 실행하면 
어드레스 자세의 여부와는 상관없이

analyze_face_pose함수 부분에서 측정된 거리 값이 안나오고 예외 처리 구문만 출력됨.

그래서 이걸 고쳐볼 파일입니다.

In [13]:
import tensorflow as tf
import numpy as np
import cv2
import time

# TensorFlow Lite 모델 로드
interpreter = tf.lite.Interpreter(model_path='movenet_lighting_tflite_float16.tflite')
interpreter.allocate_tensors()

# 전역 변수로 시작 자세의 키포인트를 저장
start_pose_keypoints = None

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# 수정된 analyze_face_pose 함수
def analyze_face_pose(current_keypoints):
    global start_pose_keypoints
    if start_pose_keypoints is None or len(start_pose_keypoints[0][0]) < 5:
        print("시작 자세가 설정되지 않았거나, 필요한 키포인트 정보가 부족합니다.")
        return

    try:
        current_left_ear = current_keypoints[0][0][3][:2]
        current_right_ear = current_keypoints[0][0][4][:2]
        start_left_ear = start_pose_keypoints[0][0][3][:2]
        start_right_ear = start_pose_keypoints[0][0][4][:2]

        current_ears_midpoint = (current_left_ear + current_right_ear) / 2
        start_ears_midpoint = (start_left_ear + start_right_ear) / 2

        height_change = current_ears_midpoint[1] - start_ears_midpoint[1]
        print("------------------------------------")
        print(f"높이 변화: {height_change:.2f}")

        start_angle = calculate_angle(start_left_ear, start_right_ear)
        current_angle = calculate_angle(current_left_ear, current_right_ear)
        angle_change = current_angle - start_angle
        print(f"각도 변화: {angle_change:.2f}도")

    except IndexError as e:
        print("예외가 발생했습니다: 필요한 키포인트 정보를 얻을 수 없습니다.", e)

def calculate_angle(p1, p2):
    angle = np.arctan2(p2[1] - p1[1], p2[0] - p1[0]) * 180.0 / np.pi
    return angle

# 웹캠에서 이미지 캡처
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 이미지 전처리
    img = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.uint8)

    # 모델 실행
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

    # 키포인트 추출 및 조정
    keypoints = np.squeeze(np.multiply(keypoints_with_scores, [frame.shape[0], frame.shape[1], 1]))

    # 얼굴 키포인트 분석을 위한 시작 자세 설정
    if start_pose_keypoints is None:
        start_pose_keypoints = keypoints_with_scores.copy()

    # 얼굴 포즈 분석
    analyze_face_pose(keypoints_with_scores)

    # 화면에 결과 표시
    draw_keypoints(frame,keypoints_with_scores, 0.4)
    time.sleep(0.2)
    
    cv2.imshow('MoveNet Lightning', frame)
    # 'q' 키를 누르면 루프에서 탈출
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


------------------------------------
높이 변화: 0.00
각도 변화: 0.00도
------------------------------------
높이 변화: -0.00
각도 변화: 0.87도
------------------------------------
높이 변화: -0.00
각도 변화: 0.73도
------------------------------------
높이 변화: -0.01
각도 변화: -1.22도
------------------------------------
높이 변화: -0.01
각도 변화: -0.44도
------------------------------------
높이 변화: -0.01
각도 변화: 0.06도
------------------------------------
높이 변화: -0.01
각도 변화: 0.95도
------------------------------------
높이 변화: -0.01
각도 변화: 1.23도
------------------------------------
높이 변화: -0.01
각도 변화: 2.49도
------------------------------------
높이 변화: -0.01
각도 변화: 2.54도
------------------------------------
높이 변화: -0.01
각도 변화: 2.52도
------------------------------------
높이 변화: -0.01
각도 변화: 2.26도
------------------------------------
높이 변화: -0.01
각도 변화: 2.17도
------------------------------------
높이 변화: -0.01
각도 변화: 2.30도
------------------------------------
높이 변화: -0.01
각도 변화: 2.22도
------------------------------------
높이 변화: -0.01
각도 변


이 코드는 TensorFlow Lite를 사용하여 웹캠에서 얻은 이미지 프레임을 실시간으로 처리합니다. 주요 기능은 다음과 같습니다:

1. **웹캠에서 이미지 캡처:** 사용자의 움직임을 실시간으로 캡처합니다.
2. **이미지 전처리 및 모델 실행:** 캡처된 이미지를 전처리하고, TensorFlow Lite 모델을 사용해 포즈 추정을 수행합니다.
3. **키포인트 추출 및 조정:** 모델에서 얻은 키포인트를 현재 프레임의 크기에 맞게 조정합니다.
4. **얼굴 키포인트 분석:** 첫 번째 프레임에서 얻은 키포인트를 시작 자세로 설정하고, 이후 프레임에서 얼굴의 높이 및 각도 변화를 분석합니다.
5. **결과 표시:** 분석 결과를 사용자에게 시각적으로 표시합니다.

이 코드를 실행하기 전에, `movenet_lighting_tflite_float16.tflite` 모델 파일이 코드와 같은 디렉토리에 있어야 합니다. 이 실험은 얼굴의 키포인트가 잘 잡히고, 그에 따라 높이와 각도가 잘 계산되는지 확인하는 것을 목표로 합니다. 실험을 통해 얼굴 키포인트의 정확한 추출과 변화 분석을 테스트할 수 있습니다.
